# Dynamic Topic Model

A dynamic topic model is 
we thought that this would be good to use because

In [31]:
import gensim
import pandas as pd
import numpy as np
from gensim.models import ldaseqmodel
from ast import literal_eval
from gensim.models.coherencemodel import CoherenceModel
import statistics

In [12]:
df = pd.read_csv("../data/processed/formatted_df.csv").drop(columns = ['Unnamed: 0'])
df

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Name,Status,Description,References,Phase,Votes,Comments
0,CVE-1999-0001,Candidate,"['ipinputc', 'bsdderived', 'tcpip', 'implement...",BUGTRAQ:19981223 Re: CERT Advisory CA-98.13 - ...,Modified (20051217),"MODIFY(1) Frech | NOOP(2) Northcutt, W...",Christey> A Bugtraq posting indicates that the...
1,CVE-1999-0002,Entry,"['buffer', 'overflow', 'nfs', 'mountd', 'give'...",BID:121 | URL:http://www.securityfocus.com...,NaN,NaN,NaN
2,CVE-1999-0003,Entry,"['execute', 'command', 'root', 'buffer', 'over...",BID:122 | URL:http://www.securityfocus.com...,NaN,NaN,NaN
3,CVE-1999-0004,Candidate,"['mime', 'buffer', 'overflow', 'email', 'clien...",CERT:CA-98.10.mime_buffer_overflows | MS:M...,Modified (19990621),"ACCEPT(8) Baker, Cole, Collins, Dik, Landfi...","Frech> Extremely minor, but I believe e-mail i..."
4,CVE-1999-0005,Entry,"['arbitrary', 'command', 'execution', 'imap', ...",BID:130 | URL:http://www.securityfocus.com...,NaN,NaN,NaN
...,...,...,...,...,...,...,...
166896,CVE-2021-46482,Candidate,"['jsish', 'v', 'discover', 'contain', 'heap', ...",MISC:https://github.com/pcmacdon/jsish/issues/66,Assigned (20220124),None (candidate not yet proposed),NaN
166897,CVE-2021-46483,Candidate,"['jsish', 'v', 'discover', 'contain', 'heap', ...",MISC:https://github.com/pcmacdon/jsish/issues/62,Assigned (20220124),None (candidate not yet proposed),NaN
166898,CVE-2021-46559,Candidate,"['firmware', 'moxa', 'tn', 'device', 'weak', '...",MISC:https://www.moxa.com/en/support/product-s...,Assigned (20220126),None (candidate not yet proposed),NaN
166899,CVE-2021-46560,Candidate,"['firmware', 'moxa', 'tn', 'device', 'allow', ...",MISC:https://www.moxa.com/en/support/product-s...,Assigned (20220126),None (candidate not yet proposed),NaN


Converts Description from string to list

In [13]:
df['Description'] = df['Description'].apply(literal_eval)

In [14]:
desc = df['Description']

In [22]:
names = df['Name']
year = []
for instance in names:
    year.append(int(instance[4:8]))
year_count = [0]
for i in range(23):
    if i == 0:
        year_count.append(year.count(i+1999))
    else:
        year_count.append(year.count(i+1999) + year_count[i]) 
print(year_count)

[0, 1541, 2778, 4313, 6663, 8161, 10794, 15380, 22238, 28578, 35549, 40436, 45428, 50015, 55416, 61536, 69815, 77731, 86931, 101250, 116731, 132000, 149784, 166901]


In [26]:
vocab = gensim.corpora.Dictionary(desc)
doc_word_matrix_array = []
for i in range(23):
    doc_word_matrix_array.append([vocab.doc2bow(doc) for doc in desc[year_count[i]:year_count[i+1]]]) 
print(doc_word_matrix_array[1])

[[(0, 1), (1, 1), (3, 1), (6, 1), (11, 1), (12, 1), (27, 1), (42, 1), (349, 1), (3095, 1), (3096, 1)], [(0, 1), (1, 1), (11, 1), (15, 1), (20, 1), (23, 1), (25, 1), (42, 1), (349, 1), (2044, 1), (3097, 1)], [(0, 1), (15, 1), (20, 1), (41, 1), (42, 1), (51, 1), (77, 1), (81, 1), (84, 1), (186, 1), (189, 1), (1626, 1), (3098, 1)], [(1, 1), (11, 1), (68, 1), (95, 1), (132, 1), (243, 1), (443, 1), (464, 1), (465, 1), (838, 1), (1090, 1), (2044, 1), (3097, 1)], [(0, 1), (41, 1), (51, 1), (77, 1), (81, 1), (84, 1), (197, 1), (258, 1), (360, 1), (3099, 1)], [(0, 1), (35, 1), (68, 2), (70, 1), (77, 1), (81, 1), (243, 1), (741, 1), (1339, 1), (3100, 1)], [(0, 1), (1, 1), (6, 1), (11, 1), (12, 1), (14, 1), (94, 1), (508, 1), (537, 1), (822, 1), (1367, 1), (2950, 1), (2951, 1), (3101, 1)], [(0, 1), (46, 1), (77, 1), (81, 1), (181, 1), (243, 1), (844, 1), (927, 1), (3102, 1), (3103, 1)], [(0, 1), (23, 1), (25, 1), (35, 1), (51, 2), (77, 1), (81, 1), (186, 1), (189, 1), (267, 1), (587, 1), (1615, 1

In [27]:
print(doc_word_matrix[3])

[(2, 1), (4, 1), (8, 1), (14, 1), (21, 1), (22, 1), (35, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1)]


We need to separate the data into each year and apply LDA individually.  

We will use that the year of the vulnerability is the 4-8 characters of each of the CVE names. Therefore, we will extract these and count the instances for each one.

See how the chain variance affects things?

In [28]:
LDA = gensim.models.ldamodel.LdaModel

In [36]:
coherence=-1000
for topics in range(5,30):
    ldamodels=[]
    ldacoherence=[]
    for i in range(23):
        ldamodels.append(LDA(corpus=doc_word_matrix_array[i], id2word=vocab, num_topics=topics))
        ldacoherence.append(CoherenceModel(model=ldamodels[i], corpus=doc_word_matrix_array[i], dictionary=vocab, coherence='u_mass').get_coherence())
    temp_coherence = statistics.mean(ldacoherence)
    #if the new model works better than previous ones then save its parameters
    if temp_coherence > coherence:
        coherence = temp_coherence
        models = ldamodels
    print(coherence)

-3.301880706875823
-3.301880706875823
-3.301880706875823
-3.301880706875823
-3.301880706875823
-3.301880706875823
-3.301880706875823
-3.301880706875823
-3.301880706875823
-3.301880706875823
-3.301880706875823
-3.301880706875823
-3.301880706875823
-3.301880706875823
-3.301880706875823
-3.301880706875823
-3.301880706875823
-3.301880706875823
-3.301880706875823
-3.301880706875823
-3.301880706875823
-3.301880706875823
-3.301880706875823


KeyboardInterrupt: 

In [39]:
perplexity=1000
for topics in range(5,30):
    ldamodels=[]
    ldaperplexity=[]
    for i in range(23):
        ldamodels.append(LDA(corpus=doc_word_matrix_array[i], id2word=vocab, num_topics=topics))
        ldaperplexity.append(ldamodels[i].log_perplexity(doc_word_matrix_array[i]))
    temp_perplexity = statistics.mean(ldaperplexity)
    #if the new model works better than previous ones then save its parameters
    if temp_perplexity < perplexity:
        perplexity = temp_perplexity
        models = ldamodels
    print(perplexity)

-7.956281010109834
-7.962120169890115
-8.032094196523879
-8.144313162116587
-8.215549825753373
-8.358429456264396
-8.464358942287353
-8.58034686195365
-8.705437203264948
-8.819712166104335


KeyboardInterrupt: 

tried coherence but it will always decrease over time
tried perplexity but it will always decrease over time
need to be able to account for wanting as few topics as possible but with min overlap